In [1]:
#command line arguments to install follwing dependencies 
!brew install cmake 
!pip install dlib 
!pip install face_recognition
!pip install playsound

/bin/bash: brew: command not found
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=76d83ffeb11f69fc993f2e9c01ae269822988d90e66cfb14911948c153046e7b
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=83c0fb4e592f69e8c61ff0d3e113ce88c52e8d1c38c6b81f979859f1310d7fa2
  Stored in directory: /root/.cache/pip/wheels/ba/f8/bb/ea57c0146b664dca3a0ada4199b0ecb5f9dfcb7b7e22b65ba2
Successfully built playsound


In [2]:
#upload the archived closed data folder to google colab and unzip it.
!unzip  /content/dataset_B_FacialImages_highResolution.zip -d /content/closed_eye


unzip:  cannot find or open /content/dataset_B_FacialImages_highResolution.zip, /content/dataset_B_FacialImages_highResolution.zip.zip or /content/dataset_B_FacialImages_highResolution.zip.ZIP.


In [ ]:
#upload the archived open data folder to google colab and unzip it.
!unzip  /content/open_eye.zip -d /content/open_eye_


Archive:  /content/open_eye.zip
   creating: /content/open_eye_/Abba_Eban/
  inflating: /content/open_eye_/Abba_Eban/Abba_Eban_0001.jpg  
   creating: /content/open_eye_/Abbas_Kiarostami/
  inflating: /content/open_eye_/Abbas_Kiarostami/Abbas_Kiarostami_0001.jpg  
   creating: /content/open_eye_/Abdel_Aziz_Al-Hakim/
  inflating: /content/open_eye_/Abdel_Aziz_Al-Hakim/Abdel_Aziz_Al-Hakim_0001.jpg  
   creating: /content/open_eye_/Abdel_Madi_Shabneh/
  inflating: /content/open_eye_/Abdel_Madi_Shabneh/Abdel_Madi_Shabneh_0001.jpg  
   creating: /content/open_eye_/Abdel_Nasser_Assidi/
  inflating: /content/open_eye_/Abdel_Nasser_Assidi/Abdel_Nasser_Assidi_0001.jpg  
  inflating: /content/open_eye_/Abdel_Nasser_Assidi/Abdel_Nasser_Assidi_0002.jpg  
   creating: /content/open_eye_/Abdoulaye_Wade/
  inflating: /content/open_eye_/Abdoulaye_Wade/Abdoulaye_Wade_0001.jpg  
  inflating: /content/open_eye_/Abdoulaye_Wade/Abdoulaye_Wade_0002.jpg  
  inflating: /content/open_eye_/Abdoulaye_Wade/Abdoul

In [ ]:

from PIL import Image, ImageDraw
import face_recognition
import os
def crop_open_eye(open_eye_dataset):
    #Declaring count variable to know the number of proccesed images.
    count = 0

    # Open eye data has many folder,so we need to iterate them.
    for person in os.listdir(open_eye_dataset):
    # Each folder has a name assoicated with it and Each folder has many images.SO we need to iterate through every subfolder.
        for img_file in os.listdir(open_eye_dataset + '/' + person):
            # Open the file using Image library/.
            im=Image.open(open_eye_dataset + '/' + person+'/'+img_file)
            # Apply facical recognition to the image file.
            open_eye_face = face_recognition.load_image_file(open_eye_dataset + '/' + person + '/' + img_file)
            # Facial cordinates are stored in the list.
            face_cordinates = face_recognition.face_landmarks(open_eye_face)
          
            # Eyes will be recongzied and stored in the below list.
            EYES = []
            try:
              # append left eye and right eye to the above list.
                EYES.append(face_cordinates[0]['left_eye'])
                EYES.append(face_cordinates[0]['right_eye'])
            except:
                continue
            # Maximum and Mininum are found for each eye.
            for eye in EYES:
                max_x = max([coordinate[0] for coordinate in eye])
                min_x = min([coordinate[0] for coordinate in eye])
                max_y = max([coordinate[1] for coordinate in eye])
                min_y = min([coordinate[1] for coordinate in eye])
              # Finally x and y ranges are found.    
                xrange = -(min_x-max_x)
                yrange = -(min_y-max_y)
              
                
                if xrange > yrange:
                    #Four boundaries are set to crop out the eye image from the face.
                    right_boundary = round(.5*xrange-5+5) + max_x
                    left_boudnary = min_x - round(.5*xrange-5+5)
                    bottom_boundary = round(((right_boundary-left_boudnary) - yrange-5+5))/2 + max_y
                    top_boundary = min_y - round(((right_boundary-left_boudnary) - yrange+8-8))/2
                else:
                    #Four boundaries are set to crop out the eye image from the face.
                    bottom_boundary = round(.5*y_range) + max_y
                    top_boundary = min_y - round(.5*y_range)
                    right_boundary = round(((bottom_boundary-top_boundary) - xrange))/2 + max_x
                    left_boudnary = min_x - round(((bottom_boundary-top_boundary) - yrange))/2
              
                #Using above the ranges ,lets crop the eye from the face
                im = Image.open(open_eye_dataset + '/' + person + '/' + img_file)
                im = im.crop((left_boudnary, top_boundary, right_boundary, bottom_boundary))
                
                # Resize image to a uniform size to sent into a model.
                im = im.resize((80,80))
                # Create a name for the folder to which the cropped image to be stored
                name = '/content/preprocessed_open_eye/' + str(count) + '.jpg'
                # save the image in a jpeg format.
                im.save(name, 'JPEG')
                count += 1
                


In [ ]:

# imports:
from PIL import Image, ImageDraw
import face_recognition
import os
def closed_eye_cropper(closed_eye):
    # Establish count for iterative file saving
    count = 0

    # For loop going through each image file
    for person in os.listdir(closed_eye):
        for img_file in os.listdir(closed_eye + '/' + person):
            # Open the file using Image library/.
            im=Image.open(closed_eye + '/' + person+'/'+img_file)
            # Apply facical recognition to the image file.
            open_eye_face = face_recognition.load_image_file(closed_eye + '/' + person + '/' + img_file)
            # Facial cordinates are stored in the list.
            face_cordinates = face_recognition.face_landmarks(open_eye_face)
          
            # Eyes will be recongzied and stored in the below list.
            EYES = []
            try:
              # append left eye and right eye to the above list.
                EYES.append(face_cordinates[0]['left_eye'])
                EYES.append(face_cordinates[0]['right_eye'])
            except:
                continue
            # Maximum and Mininum are found for each eye.
            for eye in EYES:
                max_x = max([coordinate[0] for coordinate in eye])
                min_x = min([coordinate[0] for coordinate in eye])
                max_y = max([coordinate[1] for coordinate in eye])
                min_y = min([coordinate[1] for coordinate in eye])
              # Finally x and y ranges are found.    
                xrange = -(min_x-max_x)
                yrange = -(min_y-max_y)
              
                
                if xrange > yrange:
                    #Four boundaries are set to crop out the eye image from the face.
                    right_boundary = round(.5*xrange-5+5) + max_x
                    left_boudnary = min_x - round(.5*xrange-5+5)
                    bottom_boundary = round(((right_boundary-left_boudnary) - yrange-5+5))/2 + max_y
                    top_boundary = min_y - round(((right_boundary-left_boudnary) - yrange+8-8))/2
                else:
                    #Four boundaries are set to crop out the eye image from the face.
                    bottom_boundary = round(.5*y_range) + max_y
                    top_boundary = min_y - round(.5*y_range)
                    right_boundary = round(((bottom_boundary-top_boundary) - xrange))/2 + max_x
                    left_boudnary = min_x - round(((bottom_boundary-top_boundary) - yrange))/2
              
                #Using above the ranges ,lets crop the eye from the face
                im = Image.open(closed_eye + '/' + person + '/' + img_file)
                im = im.crop((left_boudnary, top_boundary, right_boundary, bottom_boundary))
                
                # Resize image to a uniform size to sent into a model.
                im = im.resize((80,80))
                # Create a name for the folder to which the cropped image to be stored
                name = '/content/preprocessed_closed_eye/' + str(count) + '.jpg'
                # save the image in a jpeg format.
                im.save(name, 'JPEG')
                count += 1
                # print count every 200 photos to monitor progress
                if count<100:
                  pass
                else:
                  break
               
    


In [ ]:
#Preproccesing Data

crop_open_eye('open_eye_')     
closed_eye_cropper('closed_eye')

**Phase 3 ** 

1.Unzip the open eyes and closed eyes folder.

In [3]:
#!zip -r '/content/preprocessed_closed_eye.zip' 'preprocessed_closed_eye'
#!zip -r '/content/preprocessed_open_eye.zip' 'preprocessed_open_eye'

!unzip  '/content/preprocessed_closed_eye.zip' -d 'preprocessed_closed_eye'
!unzip  '/content/preprocessed_open_eye.zip' -d 'preprocessed_open_eye'



Archive:  /content/preprocessed_closed_eye.zip
   creating: preprocessed_closed_eye/content/preprocessed_closed_eye/
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/641.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/49.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/439.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/319.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/1161.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/890.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/78.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/1011.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/296.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/422.jpg  
  inflating: preprocessed_closed_eye/content/preprocessed_closed_eye/365.jpg  
  inflating: p

**PHASE 3**

2. For each image in the folder convert them into equalivalent pixel level data which has a size of 80*80 .



In [ ]:
!pip install opencv-python


In [6]:
import os
from PIL import Image, ImageDraw
import face_recognition
import os
import cv2 as c_v

def loading_images(folder, eyes = 0):

    images = []
    for filename in os.listdir(folder):
        try:
            img = c_v.imread(os.path.join(folder,filename))
            #image resizing 
            img = c_v.resize(img, (80,80)) 
            images.append([img, eyes])
        except:
            continue
        

    return images

#Retreving images from eyes open dataset. and converting it into a list
open_eyes = loading_images("/content/preprocessed_closed_eye/content/preprocessed_closed_eye", 0)
#Retreving images from eyes closed dataset. and converting it into a list
closed_eyes = loading_images("/content/preprocessed_open_eye/content/preprocessed_open_eye", 1)
eyes = closed_eyes + open_eyes

**Phase 3**


3.Spliting the whole data into X_train,X_test,y_train,y_test using treain_test_spilt with a ration of 80 /20

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

X = [] 
y = [] 
for features, label in eyes: 
     X.append(features)
     y.append(label)
X = (np.array(X).reshape(-1, 80, 80, 3))/255.0
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)


**Phase 4**


In [8]:
# Instantiate the model
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import tensorflow as tf





model = Sequential()

# Adding first three convolutional layers
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu', # activation function 
                input_shape = (80,80,3) # shape of input (image)
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))

# Adding pooling after convolutional layers
model.add(MaxPooling2D(pool_size = (2,2))) # Dimensions of the region that you are pooling

# Adding second set of convolutional layers
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
# Adding second set of convolutional layers
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))


# Add last pooling layer.
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Adding first dense layer with 256 nodes
model.add(Dense(256, activation='relu'))

# Adding a dropout layer to avoid overfitting
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3)) 

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# adding output layer
model.add(Dense(1, activation = 'sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=[tf.keras.metrics.AUC(curve = 'PR')])

# fitting the model
model.fit(X_train,
            y_train,
            batch_size=500,
            validation_data=(X_test, y_test),
            epochs=30)

# evaluate the model 
model.evaluate(X_test, y_test, verbose=1)

Epoch 1/30
4/4 [==============================] - 40s 1s/step - loss: 0.6282 - auc: 0.0466 - val_loss: 0.1718 - val_auc: 0.1042
Epoch 2/30
4/4 [==============================] - 2s 428ms/step - loss: 0.2665 - auc: 0.0652 - val_loss: 0.2181 - val_auc: 0.1061
Epoch 3/30
4/4 [==============================] - 2s 427ms/step - loss: 0.2075 - auc: 0.0855 - val_loss: 0.1922 - val_auc: 0.1056
Epoch 4/30
4/4 [==============================] - 2s 425ms/step - loss: 0.2021 - auc: 0.0689 - val_loss: 0.1787 - val_auc: 0.1061
Epoch 5/30
4/4 [==============================] - 2s 429ms/step - loss: 0.2018 - auc: 0.0713 - val_loss: 0.1696 - val_auc: 0.1100
Epoch 6/30
4/4 [==============================] - 2s 430ms/step - loss: 0.1864 - auc: 0.0837 - val_loss: 0.1759 - val_auc: 0.1127
Epoch 7/30
4/4 [==============================] - 2s 430ms/step - loss: 0.1914 - auc: 0.0726 - val_loss: 0.1673 - val_auc: 0.1142
Epoch 8/30
4/4 [==============================] - 2s 426ms/step - loss: 0.1849 - auc: 0.0850

[0.08342602849006653, 0.8100980520248413]